## Infant filter testing
This shows how to test the Infant SN filter on fixed sets of alerts from summer 2020, and how the resulting alerts can subsequently be displayed in a (local) Skyportal instance. Look at https://github.com/AmpelProject/Ampel-notebooks/tree/master/ztf/skyportal for instructions on how to download and load a Skyportal docker container.

In [ ]:
from ampel.contrib.weizmann.t0.InfantFilter import InfantFilter
from ampel.log.AmpelLogger import AmpelLogger, LogRecordFlag
from ampel.ztf.dev.DevAlertProcessor import DevAlertProcessor
from ampel.ztf.dev.DevSkyPortalClient import DevSkyPortalClient
import os

In [ ]:
logger = AmpelLogger.get_logger(console={"level": LogRecordFlag.WARNING})

We have prepared three different alert collections for use with channel development. To make this managable, all transients mached to Gaia stars have been pre-cut. Beyond this, three further subsets exist:
- HU_RAPID|HU_RAPID_SINGLE.tar.gz allows single detection alerts to pass, but only with very strict image quality constraints.
- OKC_LENSED|HU_NEARBY_CLUSTERS|TRANSIENTS_IN_ELLIPTICAL.tar.gz contains alerts found close to known potential lens sources.
- HU_TNS_PARTNER.tar.gz is a general purpose detection stream for extragalactic transients. It requires at least two detections. 

In [ ]:
def get_tarball(fname):
    """
    Fetch tarball from DESY storage
    """
    if not os.path.exists(fname):
        try:
            with requests.get(
                "https://desycloud.desy.de/index.php/s/bHqersp5xDqspNW/download",
                params={"files": fname},
                stream=True
            ) as response:
                response.raise_for_status()
                with tqdm.tqdm(
                    total=int(response.headers['Content-Length']),
                    mininterval=1,
                    unit='byte',
                    unit_scale=True,
                ) as bar:
                    with open(fname, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=1<<14):
                            f.write(chunk)
                            bar.update(len(chunk))
        except:
            try:
                os.unlink(fname)
            except:
                ...
            raise
    return fname

We next intiate a copy of the InfantFilter. All the usual parameters are available *except* the Gaia matching, `gaia_rs` has to be set to zero. Note that Gaia-based rejection has already been done. 

In [ ]:
t0filter =     InfantFilter(
        min_drb=0.3,
        min_ndet=2,
        min_tspan=4.5,
        max_ndet=99,
        max_tul=2,
        min_fwhm=0.5,
        max_tspan = 50,
        min_gal_lat=15,
        min_rb = 0.3,
        max_fwhm = 5.,
        max_elong = 100,
        max_magdiff = 0.75,
        max_nbad = 5,
        min_sso_dist = 10,
        gaia_rs = 0,
        gaia_pm_signif = 3,
        gaia_plx_signif= 3,
        gaia_veto_gmag_min= 9,
        gaia_veto_gmag_max= 20,
        gaia_excessnoise_sig_max= 999,
        ps1_sgveto_rad= 2,
        ps1_sgveto_th= 0.76,
        ps1_confusion_rad= 3,
        ps1_confusion_sg_tol= 0.1,        
        logger=logger
    )

In [ ]:
# Initialize an alert processor based on this filter
ap = DevAlertProcessor( t0filter,  include_cutouts=False )

In [ ]:
# Select tarfile
#tarfile = 'HU_RAPID|HU_RAPID_SINGLE.tar.gz'
tarfile = 'HU_TNS_PARTNER.tar.gz'

Next the real alert processing will occur. The number of alerts processed is controlled by the 'iter_max' parameter. Start with a small number for first tests. Traversing the full alert collection can be timeconsuming.

In [ ]:
# Using the rapid preselection
n_processed = ap.process_tar(
    get_tarball(tarfile), iter_max=1000
)


In [ ]:
print('Processed %s alerts out of which %s were accepted'%(n_processed,len(ap.get_accepted_alerts())))

The next step is to upload these alerts to a SkyPortal instance. The first step is to obtain a (root) token, after which a client will upload the alerts.

In [ ]:
# The first method is to go through the frontend:
# Switch to admin through http://localhost:5000/become_user/1
# copy the root token
#
# Otherwise through terminal 
# If you can access docker client through notebook (typically requires root)
# root_token = next(iter(yaml.safe_load(io.BytesIO(subprocess.check_output(["docker", "exec", "skyportal_web_1", "cat", ".tokens.yaml"]))).values()))
# Otherwise edit this to match output of:
# 'sudo docker exec skyportal_web_1 cat .tokens.yaml'
root_token = "45566190-de3d-4e7d-ac31-1f89600bc0c8"
root_token

In [ ]:
client = DevSkyPortalClient(root_token=root_token)

In [ ]:
for alert in ap.get_accepted_alerts():
    client.post_alert(alert)